In [1]:
# General libraries
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from matplotlib import cm
from mpl_toolkits.axes_grid1 import ImageGrid
import math

# Deep learning libraries
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import PIL



In [13]:
#Image generators for importing data and providing validation split for training
train_image = ImageDataGenerator(rescale=1/255,
                                 width_shift_range=0.05,
                                 height_shift_range=0.05,
                                 horizontal_flip=True,
                                 brightness_range=[0.95,1.05],
                                 validation_split=.2)

#attempting to create validation data set without augmentations applied to the training set
validation_image = ImageDataGenerator(rescale=1/255,
                                     validation_split=.2)

test_image = ImageDataGenerator(rescale=1/255)

TypeError: __init__() got an unexpected keyword argument 'seed'

In [3]:
#training data processing
train_gen = train_image.flow_from_directory(
    directory='chest_xray/train', 
    target_size=(128, 128),color_mode='grayscale',
    batch_size=32, 
    class_mode='categorical', subset='training', interpolation="lanczos",
    shuffle=True)

Found 4192 images belonging to 3 classes.


In [4]:
#validation data processing
val_gen = validation_image.flow_from_directory(
    directory='chest_xray/train', 
    target_size=(128, 128),color_mode='grayscale',
    batch_size=32, 
    class_mode='categorical', subset='validation', interpolation="lanczos",
    shuffle=True)

Found 1046 images belonging to 3 classes.


In [5]:
#testing data processing
test_gen = test_image.flow_from_directory(
    directory='chest_xray/test', 
    target_size=(128, 128), color_mode='grayscale', 
    batch_size=32, 
    class_mode='categorical', interpolation="lanczos",
    shuffle=True)

Found 624 images belonging to 2 classes.


In [6]:
print(train_gen)
print(len(train_gen))


131


In [7]:
train_gen

In [8]:
train_gen[0]

(array([[[[0.02745098],
          [0.02352941],
          [0.02352941],
          ...,
          [0.        ],
          [0.        ],
          [0.        ]],
 
         [[0.02745098],
          [0.02352941],
          [0.02352941],
          ...,
          [0.        ],
          [0.        ],
          [0.        ]],
 
         [[0.02745098],
          [0.02352941],
          [0.02352941],
          ...,
          [0.        ],
          [0.        ],
          [0.        ]],
 
         ...,
 
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.        ],
          [0.        ],
          [0.        ]],
 
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.        ],
          [0.        ],
          [0.        ]],
 
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.        ],
          [0.        ],
          [0.        ]]],
 
 
        [

In [9]:
X,y = next(train_gen)
print('x: ',type(X))
print('y: ',type(y))
print('x: ',X.shape)
print('y: ',y.shape)


x:  <class 'numpy.ndarray'>
y:  <class 'numpy.ndarray'>
x:  (32, 128, 128, 1)
y:  (32, 3)


In [10]:
#initial model
model = models.Sequential()
model.add(layers.Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(128, 128, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_gen,epochs=10,validation_data=val_gen)

Epoch 1/10
131/131 [==============================] - 146s 1s/step - loss: 0.9651 - accuracy: 0.5470 - val_loss: 0.8480 - val_accuracy: 0.6252
Epoch 2/10
131/131 [==============================] - 144s 1s/step - loss: 0.7909 - accuracy: 0.6555 - val_loss: 0.7564 - val_accuracy: 0.6759
Epoch 3/10
131/131 [==============================] - 143s 1s/step - loss: 0.7429 - accuracy: 0.6801 - val_loss: 0.6926 - val_accuracy: 0.7017
Epoch 4/10
131/131 [==============================] - 143s 1s/step - loss: 0.7165 - accuracy: 0.6944 - val_loss: 0.6872 - val_accuracy: 0.7161
Epoch 5/10
131/131 [==============================] - 143s 1s/step - loss: 0.6853 - accuracy: 0.7061 - val_loss: 0.6897 - val_accuracy: 0.7017
Epoch 6/10
131/131 [==============================] - 146s 1s/step - loss: 0.6769 - accuracy: 0.7083 - val_loss: 0.6647 - val_accuracy: 0.7228
Epoch 7/10
131/131 [==============================] - 146s 1s/step - loss: 0.6575 - accuracy: 0.7161 - val_loss: 0.6367 - val_accuracy: 0.7294

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 128)       36992     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       147584    
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

In [12]:
#2nd model iteration - increased kernel size, added padding
kernel = (8, 8)

model2 = models.Sequential()
model2.add(layers.Conv2D(filters=32,
                        kernel_size=kernel,
                        activation='relu',
                        input_shape=(128, 128, 1),
                        padding='valid'))
model2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(layers.Conv2D(128, kernel, activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(128, kernel, activation='relu'))
model2.add(layers.Flatten())
model2.add(layers.Dense(128, activation='relu'))
model2.add(layers.Dense(3, activation='softmax'))

model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model2.fit(train_gen,epochs=10,validation_data=val_gen)

Epoch 1/10
131/131 [==============================] - 196s 1s/step - loss: 1.3662 - accuracy: 0.4823 - val_loss: 0.9491 - val_accuracy: 0.5641
Epoch 2/10
131/131 [==============================] - 199s 2s/step - loss: 0.9110 - accuracy: 0.5797 - val_loss: 0.8699 - val_accuracy: 0.6004
Epoch 3/10
131/131 [==============================] - 197s 2s/step - loss: 0.8614 - accuracy: 0.6171 - val_loss: 0.8522 - val_accuracy: 0.6099
Epoch 4/10
131/131 [==============================] - 197s 2s/step - loss: 0.8169 - accuracy: 0.6386 - val_loss: 0.7743 - val_accuracy: 0.6587
Epoch 5/10
131/131 [==============================] - 195s 1s/step - loss: 0.7742 - accuracy: 0.6656 - val_loss: 0.7498 - val_accuracy: 0.6644
Epoch 6/10
131/131 [==============================] - 201s 2s/step - loss: 0.7530 - accuracy: 0.6732 - val_loss: 0.7408 - val_accuracy: 0.6730
Epoch 7/10
131/131 [==============================] - 209s 2s/step - loss: 0.7579 - accuracy: 0.6656 - val_loss: 0.7353 - val_accuracy: 0.6836

In [13]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 121, 121, 32)      2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 60, 60, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 53, 53, 128)       262272    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 19, 19, 128)       1048704   
_________________________________________________________________
flatten_1 (Flatten)          (None, 46208)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

In [14]:
#3rd model iteration - increased pool size
kernel = (8, 8)
pool = (3, 3)

model3 = models.Sequential()
model3.add(layers.Conv2D(filters=32,
                        kernel_size=kernel,
                        activation='relu',
                        input_shape=(128, 128, 1),
                        padding='valid'))
model3.add(layers.MaxPooling2D(pool_size=pool))

#hidden layers
model3.add(layers.Conv2D(128, kernel, activation='relu'))
model3.add(layers.MaxPooling2D(pool))
model3.add(layers.Conv2D(128, kernel, activation='relu'))
model3.add(layers.Flatten())
model3.add(layers.Dense(128, activation='relu'))
model3.add(layers.Dense(3, activation='softmax'))

model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model3.fit(train_gen,epochs=10,validation_data=val_gen)

Epoch 1/10
131/131 [==============================] - 146s 1s/step - loss: 1.0208 - accuracy: 0.5239 - val_loss: 0.8514 - val_accuracy: 0.6176
Epoch 2/10
131/131 [==============================] - 156s 1s/step - loss: 0.8047 - accuracy: 0.6405 - val_loss: 0.7426 - val_accuracy: 0.6778
Epoch 3/10
131/131 [==============================] - 152s 1s/step - loss: 0.7559 - accuracy: 0.6698 - val_loss: 0.7493 - val_accuracy: 0.6644
Epoch 4/10
131/131 [==============================] - 149s 1s/step - loss: 0.7150 - accuracy: 0.6823 - val_loss: 0.6908 - val_accuracy: 0.6950
Epoch 5/10
131/131 [==============================] - 149s 1s/step - loss: 0.6875 - accuracy: 0.7080 - val_loss: 0.6923 - val_accuracy: 0.7036
Epoch 6/10
131/131 [==============================] - 150s 1s/step - loss: 0.6782 - accuracy: 0.7078 - val_loss: 0.6864 - val_accuracy: 0.7017
Epoch 7/10
131/131 [==============================] - 147s 1s/step - loss: 0.6595 - accuracy: 0.7180 - val_loss: 0.6979 - val_accuracy: 0.7036

In [15]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 121, 121, 32)      2080      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 33, 33, 128)       262272    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 11, 11, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 128)         1048704   
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [16]:
#4th model iteration - changed activation function for hidden layers to tanh, converted last hidden layer to dropout layer
kernel = (8, 8)
pool = (3, 3)

model4 = models.Sequential()
model4.add(layers.Conv2D(filters=32,
                        kernel_size=kernel,
                        activation='relu',
                        input_shape=(128, 128, 1),
                        padding='valid'))
#hidden layers
model4.add(layers.MaxPooling2D(pool_size=pool))
model4.add(layers.Conv2D(128, kernel, activation='tanh'))
model4.add(layers.MaxPooling2D(pool))
model4.add(layers.Conv2D(128, kernel, activation='tanh'))
model4.add(layers.Flatten())
model4.add(layers.Dense(128, activation='tanh'))
model4.add(layers.Dropout(rate=.25))
#output layer
model4.add(layers.Dense(3, activation='softmax'))

model4.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model4.fit(train_gen,epochs=10,validation_data=val_gen)

Epoch 1/10
131/131 [==============================] - 154s 1s/step - loss: 1.2719 - accuracy: 0.3946 - val_loss: 1.0502 - val_accuracy: 0.4847
Epoch 2/10
131/131 [==============================] - 156s 1s/step - loss: 1.1969 - accuracy: 0.4010 - val_loss: 1.0627 - val_accuracy: 0.4847
Epoch 3/10
131/131 [==============================] - 150s 1s/step - loss: 1.1745 - accuracy: 0.4008 - val_loss: 1.0532 - val_accuracy: 0.4847
Epoch 4/10
131/131 [==============================] - 150s 1s/step - loss: 1.1488 - accuracy: 0.4189 - val_loss: 1.0531 - val_accuracy: 0.4847
Epoch 5/10
131/131 [==============================] - 154s 1s/step - loss: 1.1362 - accuracy: 0.4156 - val_loss: 1.0551 - val_accuracy: 0.4847
Epoch 6/10
131/131 [==============================] - 159s 1s/step - loss: 1.1161 - accuracy: 0.4303 - val_loss: 1.0542 - val_accuracy: 0.4847
Epoch 7/10
131/131 [==============================] - 165s 1s/step - loss: 1.1090 - accuracy: 0.4346 - val_loss: 1.0548 - val_accuracy: 0.4847

In [17]:
model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 121, 121, 32)      2080      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 33, 33, 128)       262272    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 11, 11, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 4, 4, 128)         1048704   
_________________________________________________________________
flatten_3 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)              

In [18]:
#5th model iteration - reverted activation function changes, kept dropout layer
kernel = (8, 8)
pool = (3, 3)

model5 = models.Sequential()
model5.add(layers.Conv2D(filters=32,
                        kernel_size=kernel,
                        activation='relu',
                        input_shape=(128, 128, 1),
                        padding='valid'))
#hidden layers
model5.add(layers.MaxPooling2D(pool_size=pool))
model5.add(layers.Conv2D(128, kernel, activation='relu'))
model5.add(layers.MaxPooling2D(pool))
model5.add(layers.Conv2D(128, kernel, activation='relu'))
model5.add(layers.Flatten())
model5.add(layers.Dense(128, activation='relu'))
model5.add(layers.Dropout(rate=.25))
#output layer
model5.add(layers.Dense(3, activation='softmax'))

#compiling and fitting the model
model5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model5.fit(train_gen,epochs=10,validation_data=val_gen)

Epoch 1/10
131/131 [==============================] - 150s 1s/step - loss: 1.0195 - accuracy: 0.5091 - val_loss: 0.9309 - val_accuracy: 0.5822
Epoch 2/10
131/131 [==============================] - 151s 1s/step - loss: 0.8633 - accuracy: 0.6081 - val_loss: 0.7986 - val_accuracy: 0.6396
Epoch 3/10
131/131 [==============================] - 155s 1s/step - loss: 0.7897 - accuracy: 0.6467 - val_loss: 0.7663 - val_accuracy: 0.6530
Epoch 4/10
131/131 [==============================] - 158s 1s/step - loss: 0.7475 - accuracy: 0.6696 - val_loss: 0.7606 - val_accuracy: 0.6511
Epoch 5/10
131/131 [==============================] - 161s 1s/step - loss: 0.7252 - accuracy: 0.6796 - val_loss: 0.7195 - val_accuracy: 0.6950
Epoch 6/10
131/131 [==============================] - 153s 1s/step - loss: 0.7017 - accuracy: 0.7068 - val_loss: 0.6799 - val_accuracy: 0.7008
Epoch 7/10
131/131 [==============================] - 161s 1s/step - loss: 0.6931 - accuracy: 0.7090 - val_loss: 0.6795 - val_accuracy: 0.7036

In [19]:
model5.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 121, 121, 32)      2080      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 33, 33, 128)       262272    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 11, 11, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 4, 4, 128)         1048704   
_________________________________________________________________
flatten_4 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)              

In [20]:
#5th model iteration - added 1 extra convlution, reduced kernel size
kernel = (8, 8)
pool = (3, 3)

model6 = models.Sequential()
model6.add(layers.Conv2D(filters=32,
                        kernel_size=kernel,
                        activation='relu',
                        input_shape=(128, 128, 1),
                        padding='valid'))
#hidden layers
model6.add(layers.MaxPooling2D(pool_size=pool))
model6.add(layers.Conv2D(128, kernel, activation='relu'))
model6.add(layers.Conv2D(128, kernel, activation='relu'))
model6.add(layers.MaxPooling2D(pool))
model6.add(layers.Conv2D(128, kernel, activation='relu'))
model6.add(layers.Flatten())
model6.add(layers.Dense(128, activation='relu'))
model6.add(layers.Dropout(rate=.25))
#output layer
model6.add(layers.Dense(3, activation='softmax'))

#compiling and fitting the model
model6.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model6.fit(train_gen,epochs=20,validation_data=val_gen)

Epoch 1/20
131/131 [==============================] - 175s 1s/step - loss: 1.0704 - accuracy: 0.4776 - val_loss: 1.0139 - val_accuracy: 0.5105
Epoch 2/20
131/131 [==============================] - 176s 1s/step - loss: 0.9390 - accuracy: 0.5580 - val_loss: 0.8497 - val_accuracy: 0.5966
Epoch 3/20
131/131 [==============================] - 178s 1s/step - loss: 0.8545 - accuracy: 0.6104 - val_loss: 0.7736 - val_accuracy: 0.6654
Epoch 4/20
131/131 [==============================] - 181s 1s/step - loss: 0.7860 - accuracy: 0.6582 - val_loss: 0.7990 - val_accuracy: 0.6558
Epoch 5/20
131/131 [==============================] - 189s 1s/step - loss: 0.7678 - accuracy: 0.6634 - val_loss: 0.7599 - val_accuracy: 0.6577
Epoch 6/20
131/131 [==============================] - 182s 1s/step - loss: 0.7566 - accuracy: 0.6718 - val_loss: 0.7462 - val_accuracy: 0.6750
Epoch 7/20
131/131 [==============================] - 182s 1s/step - loss: 0.7678 - accuracy: 0.6756 - val_loss: 0.7515 - val_accuracy: 0.6855

In [21]:
model6.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 121, 121, 32)      2080      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 33, 33, 128)       262272    
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 26, 26, 128)       1048704   
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 1, 1, 128)         1048704   
_________________________________________________________________
flatten_5 (Flatten)          (None, 128)              

# Larger Image Sizes

Due to the granularity typically involved in human interpretation of Chest X-ray images (i.e. identifying presence and patterns of infiltrate in the lungs) we are experimenting with increasing the size of the images to achieve greater precision. This poses a greater risk of overfitting, and increases training time, however, it is possible that using convolutions and pooling, our model may be able to aggregate and interpret this finer detail if tuned appropriately.

In [6]:
#training data processing using half length and width of original image size
train_gen_0 = train_image.flow_from_directory(
    directory='chest_xray/train', 
    target_size=(534, 381),color_mode='grayscale',
    batch_size=32, 
    class_mode='categorical', subset='training', interpolation="lanczos",
    shuffle=True)

#validation data processing using half length and width of original image size
val_gen_0 = validation_image.flow_from_directory(
    directory='chest_xray/train', 
    target_size=(534, 381),color_mode='grayscale',
    batch_size=32, 
    class_mode='categorical', subset='validation', interpolation="lanczos",
    shuffle=True)

Found 4192 images belonging to 3 classes.
Found 1046 images belonging to 3 classes.


In [10]:
#initial model using half length and width of original image size
model = models.Sequential()

#input layer
model.add(layers.Conv2D(filters=32,
                        kernel_size=(20, 20),
                        activation='relu',
                        input_shape=(534, 381, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

#hidden layers
model.add(layers.Conv2D(381, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (2, 2), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))

#output layer
model.add(layers.Dense(3, activation='softmax'))

#compile and compute
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'AUC'])
#due to running this overnight, I am creating a save file for this model
model_checkpoint_callback = ModelCheckpoint(filepath="./Checkpoints",
                                           save_weights_only=True,
                                           monitor='val_accuracy',
                                           mode='max',
                                           save_best_only=True)

model.fit(train_gen_0, epochs=20, validation_data=val_gen_0)

Epoch 1/20
131/131 [==============================] - 2463s 19s/step - loss: 1.0883 - accuracy: 0.6412 - auc: 0.8108 - val_loss: 0.6818 - val_accuracy: 0.6874 - val_auc: 0.8729
Epoch 2/20
131/131 [==============================] - 2502s 19s/step - loss: 0.6530 - accuracy: 0.7178 - auc: 0.8811 - val_loss: 0.5516 - val_accuracy: 0.7581 - val_auc: 0.9116
Epoch 3/20
131/131 [==============================] - 2481s 19s/step - loss: 0.5758 - accuracy: 0.7455 - auc: 0.9051 - val_loss: 0.5299 - val_accuracy: 0.7725 - val_auc: 0.9202
Epoch 4/20
131/131 [==============================] - 2490s 19s/step - loss: 0.5448 - accuracy: 0.7634 - auc: 0.9147 - val_loss: 0.5405 - val_accuracy: 0.7696 - val_auc: 0.9196
Epoch 5/20
131/131 [==============================] - 2471s 19s/step - loss: 0.5412 - accuracy: 0.7564 - auc: 0.9151 - val_loss: 0.5372 - val_accuracy: 0.7696 - val_auc: 0.9178
Epoch 6/20
  2/131 [..............................] - ETA: 19:37 - loss: 0.3319 - accuracy: 0.9062 - auc: 0.9750

KeyboardInterrupt: 